# Basic FAISS

## Notebook Summary: RAG + LLaMA-2 with Compound QA Support

This notebook extends centralized RAG-based extractive QA using a fine-tuned LLaMA-2 model to handle compound telecom questions.

### Key Enhancements Over Previous Version:

1. **Compound Question Handling**  
   Detects and splits multi-part queries using logical connectors (e.g., "and", "or"). Each clause is independently answered using the same RAG + LLaMA-2 pipeline.

2. **Multi-Answer Inference**  
   For compound queries, generates multiple precise answers and formats them as bullet-style sub-responses.

3. **Context Validation**  
   Verifies that each sub-answer appears in the retrieved context to ensure factual grounding.

4. **Evaluation**  
   Measures performance on 100 QA pairs using:
   - **Exact Match** / **F1** (SQuAD)
   - **ROUGE-L**
   - **BLEU**

This version improves robustness in handling complex telecom queries and offers more reliable QA outputs for your thesis RAG baseline.

In [1]:
from pathlib import Path
import faiss
import pickle
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer

In [2]:
# Load FAISS index and chunks
index_path = "/mnt/data/RAG/3gpp_index.faiss"
chunks_path = "/mnt/data/RAG/3gpp_chunks.pkl"

index = faiss.read_index(index_path)
with open(chunks_path, "rb") as f:
    documents = pickle.load(f)

# Load embedding model used for indexing
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
def retrieve_context(query, top_k=3):
    query_emb = embedding_model.encode([query], normalize_embeddings=True)
    D, I = index.search(query_emb.astype("float32"), top_k)
    return [documents[i] for i in I[0]]

In [3]:
SYSTEM_PROMPT = (
    "You are a precise assistant. Extract the exact answer span from the context. "
    "Do not paraphrase, summarize, or add extra information. "
    "The answer must appear exactly in the context."
)

def build_rag_prompt(context_chunks, question):
    combined_context = "\n\n".join([chunk['content'] for chunk in context_chunks])
    user_prompt = (
        f"Context: {combined_context}\n\n"
        f"Question: {question}\n"
        f"Answer from the context only:"
    )
    return f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n{user_prompt} [/INST]"

model_path = "/mnt/data/llama2_qa_lora_output5/final"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).to("cuda")

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
def clean_prediction(raw_text):
    # Remove everything before the last [INST]
    answer = raw_text.split("[/INST]")[-1].strip()

    # Remove strange characters
    answer = re.sub(r"[^\w\s\-.,:/()]", "", answer)

    # Remove repeating phrases like "The key is... The key is... The key is..."
    answer = re.sub(r'(\b.+?:)(\s*\1)+', r'\1', answer)

    # Trim repetitive word loops (e.g., "structured as follows" x 5)
    tokens = answer.split()
    for i in range(1, len(tokens) // 2):
        if tokens[:i] == tokens[i:2*i]:
            answer = " ".join(tokens[:i])
            break

    # Optionally truncate to sentence boundary
    sentence_end = re.search(r'[.?!]', answer)
    if sentence_end:
        answer = answer[:sentence_end.end()]

    return answer.strip()

In [5]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk

STOPWORDS = set(stopwords.words("english"))

import re

def normalize(text):
    return re.sub(r'\W+', ' ', text.lower())

def lexical_overlap(query, chunk):
    q_tokens = set(normalize(query).split()) - STOPWORDS
    c_tokens = set(normalize(chunk).split()) - STOPWORDS
    return len(q_tokens & c_tokens) / (len(q_tokens | c_tokens) + 1e-5)

def tfidf_score(query, chunk, vectorizer=None):
    docs = [query, chunk]
    if not vectorizer:
        vectorizer = TfidfVectorizer().fit(docs)
    vecs = vectorizer.transform(docs)
    return (vecs[0] @ vecs[1].T).A[0][0]

In [6]:
def rerank_chunks(chunks, query, alpha_overlap=0.7, beta_faiss=0.3, top_k=3):
    vectorizer = TfidfVectorizer().fit([query] + [c["content"] for c in chunks])
    reranked = []

    for idx, c in enumerate(chunks):
        overlap = lexical_overlap(query, c["content"])
        tfidf_sim = tfidf_score(query, c["content"], vectorizer)
        faiss_rank_bonus = (len(chunks) - idx) / len(chunks)

        # Final rerank score = weighted combination
        score = alpha_overlap * overlap + (1 - alpha_overlap) * tfidf_sim + beta_faiss * faiss_rank_bonus

        reranked.append((score, c))

    reranked.sort(reverse=True, key=lambda x: x[0])
    return [c for _, c in reranked[:top_k]]

In [7]:
def split_compound_question(q):
    parts = re.split(r"\band\b|\bor\b|[,;]", q)
    return [p.strip() for p in parts if len(p.strip().split()) > 3]


def answer_with_rag_llama(question, top_k=5, verbose=False):
    initial_chunks = retrieve_context(question, top_k=10)  # Increase FAISS recall
    retrieved = rerank_chunks(initial_chunks, question, top_k=top_k)  # Rerank + filter

    sub_qs = split_compound_question(question)

    # Handle compound question (multi-prompt)
    if len(sub_qs) > 1:
        answers = []
        for sq in sub_qs:
            sub_prompt = build_rag_prompt(retrieved, sq)
            raw = qa_pipeline(
                sub_prompt, 
                max_new_tokens=160, 
                do_sample=False, 
                eos_token_id=tokenizer.eos_token_id, 
                pad_token_id=tokenizer.eos_token_id
            )[0]["generated_text"]

            ans = clean_prediction(raw)
            answers.append(f"→ {sq}: {ans}")

        full_answer = "\n".join(answers)

        # Context containment check (on full answer)
        all_context = " ".join([c["content"] for c in retrieved])
        if not any(ans.split(": ", 1)[-1] in all_context for ans in answers):
            print("🚨 One or more sub-answers not found in context — check retrieval or generation.")
        return full_answer, retrieved

    # Handle simple (single-clause) question
    prompt = build_rag_prompt(retrieved, question)
    raw_output = qa_pipeline(
        prompt, 
        max_new_tokens=160, 
        do_sample=False, 
        eos_token_id=tokenizer.eos_token_id, 
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]

    answer = clean_prediction(raw_output)

    # Sanity check
    if len(answer.split()) < 2 or len(answer.split()) > 40:
        print("⚠️ Warning: Possibly bad output. Check content or retrieval.")

    # Context containment validation
    all_context = " ".join([c["content"] for c in retrieved])
    if answer not in all_context:
        print("🚨 Answer not found in retrieved context — check prompt or retrieval quality.")

    if verbose:
        print("📌 Prompt:\n", prompt)
        print("\n🧾 Raw Output:\n", raw_output)
        print("\n✅ Cleaned Answer:", answer)
        for i, chunk in enumerate(retrieved):
            print(f"\n--- Context {i+1} ---")
            print(chunk["content"])

    return answer, retrieved

In [8]:
import json
from tqdm import tqdm
from evaluate import load

# Load QA pairs
def load_qa_pairs(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

qa_pairs = load_qa_pairs("3gpp_qa_100_pairs.jsonl")

# Load metrics
squad_metric = load("squad")
rouge = load("rouge")
bleu = load("bleu")

bleu_predictions = []
bleu_references = []
results = []

for sample in tqdm(qa_pairs):
    question = sample["question"]
    reference = sample["answer"]

    try:
        prediction, _ = answer_with_rag_llama(question)
    except Exception as e:
        print(f"⚠️ Error on: {question}\n{e}")
        prediction = ""

    # Add to metrics
    squad_metric.add(
        prediction={"id": str(hash(question)), "prediction_text": prediction},
        reference={"id": str(hash(question)), "answers": {"text": [reference], "answer_start": [0]}}
    )
    rouge.add(prediction=prediction, reference=reference)
    bleu_predictions.append(prediction)
    bleu_references.append([reference])
    results.append({
        "question": question,
        "reference": reference,
        "prediction": prediction
    })

# Compute final scores
squad_scores = squad_metric.compute()
rouge_scores = rouge.compute()
bleu_score = bleu.compute(predictions=bleu_predictions, references=bleu_references)["bleu"]

# Print results
print("\n📊 Final Evaluation Results (Compound QA Enabled):")
print(f"Exact Match (EM): {squad_scores['exact_match']:.2f}")
print(f"F1 Score        : {squad_scores['f1']:.2f}")
print(f"ROUGE-L         : {rouge_scores['rougeL']:.4f}")
print(f"BLEU            : {bleu_score:.4f}")

  2%|▊                                          | 2/100 [00:16<13:13,  8.10s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 Answer not found in retrieved context — check prompt or retrieval quality.


  5%|██▏                                        | 5/100 [00:29<07:30,  4.74s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 Answer not found in retrieved context — check prompt or retrieval quality.


  6%|██▌                                        | 6/100 [00:36<09:00,  5.75s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

 98%|█████████████████████████████████████████▏| 98/100 [09:23<00:13,  6.84s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 Answer not found in retrieved context — check prompt or retrieval quality.


100%|█████████████████████████████████████████| 100/100 [09:38<00:00,  5.78s/it]

🚨 Answer not found in retrieved context — check prompt or retrieval quality.



📊 Final Evaluation Results (Compound QA Enabled):
Exact Match (EM): 0.00
F1 Score        : 21.39
ROUGE-L         : 0.2242
BLEU            : 0.0231
